In [82]:
import pandas as pd
from sklearn.preprocessing import Normalizer

# -----------------------------
# 1️⃣ Create Master Lookup (Sets 1-17)
# -----------------------------
lookup_list = []

for i in range(1, 18):
    filename = f"C:/Users/migue/Downloads/set_{i}.csv"
    
    df = pd.read_csv(filename)
    df.columns = df.columns.str.strip()
    # Remove unnamed columns if any exist
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    if 'image_id' not in df.columns:
        raise ValueError(f"Set {i} missing image_id")
        
    df['image_id'] = df['image_id'].astype(str).str.strip()
    
    # Ensure unique IDs and create time sequence
    df = df.drop_duplicates(subset='image_id').reset_index(drop=True)
    df['time_dataset'] = range(1, len(df) + 1)
    df['set_id'] = i
    
    lookup_list.append(df[['set_id', 'image_id', 'time_dataset']])

lookup_df = pd.concat(lookup_list, ignore_index=True)

# -----------------------------
# 2️⃣ Load & Prep Target File (32 Embeddings)
# -----------------------------
target_path = r"C:\Users\migue\Downloads\kosala_32_images.csv"
other_file = pd.read_csv(target_path, sep=';', engine='python')

other_file.columns = other_file.columns.str.strip()
other_file['image_id'] = other_file['image_id'].astype(str).str.strip()
if 'set_id' in other_file.columns:
    other_file['set_id'] = other_file['set_id'].astype(int)

# -----------------------------
# 3️⃣ Merge Data
# -----------------------------
merged_df = other_file.merge(
    lookup_df, 
    on=['set_id', 'image_id'], 
    how='left'
)

# -----------------------------
# 4️⃣ Post-Processing
# -----------------------------

# A. One-Hot Encode 'type'
merged_df = pd.get_dummies(merged_df, columns=['type'], prefix='type', dtype=int)

# B. Select Exact Columns (Dynamic for z0-z31)
z_cols = [f'z{i}' for i in range(32)]

base_columns = [
    'set_id', 'time_dataset', 
    'type_adhesion', 'type_flank_wear', 'type_flank_wear+adhesion', 'wear'
]

# Ensure all columns exist before reindexing
desired_columns = base_columns + z_cols
final_df = merged_df.reindex(columns=desired_columns, fill_value=0)

# C. Sorting
final_df = final_df.sort_values(by=['set_id', 'time_dataset'], ascending=[True, True])

# D. Clean Numeric Columns
for col in z_cols:
    # ⚠️ CRITICAL: I commented this out. 
    # Only uncomment if your CSV uses dots as thousands separators (e.g. 1.000 = one thousand).
    # If your CSV uses dots as decimals (0.5), this line destroys your data.
    # final_df[col] = final_df[col].astype(str).str.replace('.', '', regex=False)
    
    # Convert to numeric safely
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce').fillna(0)

# E. Apply Normalization (L2)
# axis=1 normalizes each ROW (embedding vector) independently.
# This ensures the vector length is 1.
transformer = Normalizer(norm='l2')
final_df[z_cols] = transformer.fit_transform(final_df[z_cols])

# -----------------------------
# 5️⃣ Save & Verify
# -----------------------------
final_df.to_csv(
    "C:/Users/migue/Downloads/images_with_time_processed_32.csv",
    index=False,
    sep=';'
)

print(f"Processing complete. Shape: {final_df.shape}")
print("First 5 rows (z-values should now be typically between -1 and 1):")
print(final_df.iloc[:10].to_string())

Processing complete. Shape: (1479, 38)
First 5 rows (z-values should now be typically between -1 and 1):
    set_id  time_dataset  type_adhesion  type_flank_wear  type_flank_wear+adhesion   wear           z0           z1           z2           z3           z4           z5            z6           z7           z8           z9          z10          z11          z12          z13          z14          z15          z16          z17          z18          z19          z20           z21          z22           z23          z24          z25          z26           z27          z28          z29          z30           z31
18       1             6              1                0                         0   75.0 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000  0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000  0.0000000000 0.0000

In [86]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler

# [Assuming final_df exists from Steps 1-5]

# -----------------------------
# 6️⃣ Load & Process New Features File
# -----------------------------
features_path = r"C:/Users/migue/Downloads/sensor_embeddings_jesse.csv"
features_df = pd.read_csv(features_path, sep=';', engine='python')

features_df.columns = features_df.columns.str.strip()
print(f"Loaded sensor data. Rows: {len(features_df)}")

# A. Handle 'set' and 'sample_index' types
if 'set' in features_df.columns:
    features_df = features_df.dropna(subset=['set'])
    features_df['set'] = features_df['set'].astype(int)

if 'sample_index' in features_df.columns:
    features_df = features_df.dropna(subset=['sample_index'])
    features_df['sample_index'] = features_df['sample_index'].astype(int)

# 🚨 Align indices: Sensor(0) matches Image(1)
print("ℹ️ Adjusting 'sample_index' by +1 to match 'time_dataset'...")
features_df['sample_index'] = features_df['sample_index'] + 1

# B. Normalize 'n', 'Vf', 'Vc' (Standard Scaler)
process_params = ['n', 'Vf', 'Vc']
existing_params = [col for col in process_params if col in features_df.columns]

if existing_params:
    for col in existing_params:
        features_df[col] = pd.to_numeric(features_df[col], errors='coerce')
        if features_df[col].std() == 0:
            features_df[col] = 0.0
        else:
            scaler = StandardScaler()
            features_df[[col]] = scaler.fit_transform(features_df[[col]])

# C. Handle 'sample_index_scaled'
if 'sample_index_scaled' in features_df.columns:
    features_df['sample_index_scaled'] = pd.to_numeric(features_df['sample_index_scaled'], errors='coerce').fillna(0)
    features_df['sample_index_scaled'] = features_df['sample_index_scaled'] * 10
elif 'sample_index' in features_df.columns:
    mm_scaler = MinMaxScaler()
    features_df['sample_index_scaled'] = mm_scaler.fit_transform(features_df[['sample_index']]) * 10

# D. Normalize Sensor Embeddings
sens_cols = [f'sens_emb_{i}' for i in range(32)]
existing_sens_cols = [c for c in sens_cols if c in features_df.columns]

for col in existing_sens_cols:
    features_df[col] = pd.to_numeric(features_df[col], errors='coerce').fillna(0)

if existing_sens_cols:
    sens_transformer = Normalizer(norm='l2')
    features_df[existing_sens_cols] = sens_transformer.fit_transform(features_df[existing_sens_cols])

# -----------------------------
# 7️⃣ Prepare for Merge
# -----------------------------
features_df = features_df.rename(columns={'set': 'set_id', 'sample_index': 'time_dataset'})

cols_to_keep = ['set_id', 'time_dataset', 'n', 'Vf', 'material_CK45', 'wear_level', 
                'Ae', 'fz', 'Ap', 'Vc', 'sample_index_scaled'] + existing_sens_cols

final_cols_to_keep = [c for c in cols_to_keep if c in features_df.columns]
features_df_clean = features_df[final_cols_to_keep]

# -----------------------------
# 8️⃣ Merge
# -----------------------------
final_df['set_id'] = final_df['set_id'].astype(int)
final_df['time_dataset'] = final_df['time_dataset'].astype(int)

combined_df = final_df.merge(features_df_clean, on=['set_id', 'time_dataset'], how='left')

# Check for NaN explosion
if 'n' in combined_df.columns:
    nan_count = combined_df['n'].isna().sum()
    if nan_count == len(combined_df):
        print("❌ CRITICAL ERROR: Merge failed (100% NaNs).")
    else:
        print(f"Merge success. Rows with missing sensor data: {nan_count}")

# -----------------------------
# 🆕 9️⃣ Create Persistent Targets (Logic Injection)
# -----------------------------
# Define the 3 types and corresponding new target names
type_map = {
    'type_adhesion': 'target_adhesion', 
    'type_flank_wear': 'target_flank', 
    'type_flank_wear+adhesion': 'target_flank_adhesion'
}

# Ensure 'wear_level' is numeric
wear_col = 'wear_level'
combined_df[wear_col] = pd.to_numeric(combined_df[wear_col], errors='coerce').fillna(0)

print("\n--- Generating Persistent Targets ---")
for type_col, target_col in type_map.items():
    if type_col in combined_df.columns:
        # 1. Mask: Keep wear value ONLY if binary type is 1. Else NaN.
        # This initializes the 'activation' points.
        combined_df[target_col] = combined_df[wear_col].where(combined_df[type_col] == 1)
        
        # 2. Forward Fill within Set: 
        # Once activated, the value propagates forward until the set ends or a new activation overrides it.
        combined_df[target_col] = combined_df.groupby('set_id')[target_col].ffill()
        
        # 3. Fill Initial NaNs: Rows before the first activation become 0.
        combined_df[target_col] = combined_df[target_col].fillna(0)
        
        print(f"Generated {target_col}: Non-zero count = {(combined_df[target_col] > 0).sum()}")

# -----------------------------
# 🔟 Save
# -----------------------------
output_path = "C:/Users/migue/Downloads/images_with_time_and_sensors_processed.csv"
combined_df.to_csv(output_path, index=False, sep=';', float_format='%.10f')

print(f"Saved to: {output_path}")
print(combined_df[['set_id', 'time_dataset', 'wear_level'] + list(type_map.values())].head(10).to_string())

Loaded sensor data. Rows: 1390
ℹ️ Adjusting 'sample_index' by +1 to match 'time_dataset'...
Merge success. Rows with missing sensor data: 221

--- Generating Persistent Targets ---
Generated target_adhesion: Non-zero count = 918
Generated target_flank: Non-zero count = 994
Generated target_flank_adhesion: Non-zero count = 326
Saved to: C:/Users/migue/Downloads/images_with_time_and_sensors_processed.csv
   set_id  time_dataset     wear_level  target_adhesion   target_flank  target_flank_adhesion
0       1             6  45.0000000000    45.0000000000   0.0000000000           0.0000000000
1       1             8  60.0000000000    60.0000000000   0.0000000000           0.0000000000
2       1             9  60.0000000000    60.0000000000   0.0000000000           0.0000000000
3       1            11  75.0000000000    75.0000000000   0.0000000000           0.0000000000
4       1            27 105.0000000000    75.0000000000 105.0000000000           0.0000000000
5       1            28 105.00

# Using both the Image and the Sensor Embeddings

In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

# Force Pandas to display ALL columns (no "...")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 1. Define Target Columns (y)
y_cols = ['target_adhesion', 'target_flank', 'target_flank_adhesion']

# 2. Define Features (X)
# UPDATED: We now drop 'set_id' and 'time_dataset' alongside the leakage columns
drop_cols = y_cols + ['wear_level', 'wear', 'set_id', 'time_dataset']

# Safe drop (ignores if any column doesn't exist)
X = combined_df.drop(columns=[c for c in drop_cols if c in combined_df.columns])
y = combined_df[y_cols]

# -----------------------------
# 🚨 CRITICAL FIX: Handle NaNs from Merge
# -----------------------------
# Fill missing sensor data with 0 (Mean/Zero Vector)
if X.isna().sum().sum() > 0:
    print(f"⚠️ Found missing values in X. Filling with 0.")
    X = X.fillna(0)
if y.isna().sum().sum() > 0:
    y = y.fillna(0)

print(f"X Shape: {X.shape}")
print(f"y Shape: {y.shape}")

# 3. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# 4. Train Neural Network
model = MLPRegressor(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    solver='adam',
    
    # 1. LEARNING RATE
    learning_rate_init=0.0003,      # Default is 0.001. Lower = more precise but slower.
    
    # 2. OVERFITTING PROTECTION (L2 Regularization)
    alpha=0.01,                     # Default is 0.0001. Increase (e.g., 0.01, 0.1) to reduce overfitting.
    
    # 3. EARLY STOPPING
    early_stopping=True,            # Stop if validation score doesn't improve
    validation_fraction=0.1,        # Use 10% of train data to check for early stopping
    n_iter_no_change=10,            # Stop if no improvement for 10 epochs
    
    max_iter=1000,
    random_state=42,
    verbose=False
)

print("Training Neural Network...")
model.fit(X_train, y_train)

# 5. Evaluate with MAE
y_pred = model.predict(X_test)

# Calculate MAE for each individual target column
mae_per_col = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
total_mae = mean_absolute_error(y_test, y_pred)

print("\n" + "="*50)
print("📊 MODEL PERFORMANCE (Mean Absolute Error)")
print("="*50)
print(f"Global Average MAE:      {total_mae:.4f}")
print("-" * 50)
for i, col in enumerate(y_cols):
    print(f"MAE for {col:<25}: {mae_per_col[i]:.4f}")
print("="*50)

# 6. Print X Columns and Head (Checking the features)
print("\n" + "="*50)
print(f"🔍 CHECKING X FEATURES (Total: {len(X.columns)})")
print("="*50)
print("Feature Names:")
print(X.columns.tolist())
print("-" * 50)
print("First 5 Rows of X (All columns visible):")
print(X.head())

⚠️ Found missing values in X. Filling with 0.
X Shape: (1479, 75)
y Shape: (1479, 3)
Training Neural Network...

📊 MODEL PERFORMANCE (Mean Absolute Error)
Global Average MAE:      15.6634
--------------------------------------------------
MAE for target_adhesion          : 24.4282
MAE for target_flank             : 15.0868
MAE for target_flank_adhesion    : 7.4751

🔍 CHECKING X FEATURES (Total: 75)
Feature Names:
['type_adhesion', 'type_flank_wear', 'type_flank_wear+adhesion', 'z0', 'z1', 'z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11', 'z12', 'z13', 'z14', 'z15', 'z16', 'z17', 'z18', 'z19', 'z20', 'z21', 'z22', 'z23', 'z24', 'z25', 'z26', 'z27', 'z28', 'z29', 'z30', 'z31', 'n', 'Vf', 'material_CK45', 'Ae', 'fz', 'Ap', 'Vc', 'sample_index_scaled', 'sens_emb_0', 'sens_emb_1', 'sens_emb_2', 'sens_emb_3', 'sens_emb_4', 'sens_emb_5', 'sens_emb_6', 'sens_emb_7', 'sens_emb_8', 'sens_emb_9', 'sens_emb_10', 'sens_emb_11', 'sens_emb_12', 'sens_emb_13', 'sens_emb_14', 'sens_emb_15',

# Just the image embeddings for a NN


In [91]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

# 1. Define Target Columns (y)
y_cols = ['target_adhesion', 'target_flank', 'target_flank_adhesion']

# 2. Define Features to DROP (As requested)
# We strictly remove all sensors, physical params, and the answers (targets/wear_level)
cols_to_drop = [
    # Physical Params
    
    # Identifiers (Prevent memorization)
    'set_id', 'time_dataset', 'wear', 'wear_level',
    # Targets (Answers)
    'target_adhesion', 'target_flank', 'target_flank_adhesion'
]

# Add Sensor Embeddings 0-31 to the drop list
cols_to_drop += [f'sens_emb_{i}' for i in range(32)]

# 3. Create X and y
# Note: X will now contain mostly 'z0'-'z31' and the 'type_...' binary columns
X = combined_df.drop(columns=[c for c in cols_to_drop if c in combined_df.columns])
y = combined_df[y_cols]

# 4. Handle NaNs (Safety)
if X.isna().sum().sum() > 0:
    print("⚠️ Found NaNs in X (filling with 0)...")
    X = X.fillna(0)
if y.isna().sum().sum() > 0:
    y = y.fillna(0)

print(f"X Shape: {X.shape}")
print(f"y Shape: {y.shape}")

# 5. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# 6. Train Neural Network (With Regularization to fight overfitting)
model = MLPRegressor(
    hidden_layer_sizes=(128, 64), 
    activation='relu',             
    solver='adam',                 
    learning_rate_init=0.0003,      
    alpha=0.05,                    # L2 Regularization (Higher = less overfitting)
    early_stopping=True,           # Stop if validation score stalls
    validation_fraction=0.1,
    n_iter_no_change=10,
    max_iter=1000,                  
    random_state=42,
    verbose=False                   
)

print("\nTraining Image-Only Neural Network...")
model.fit(X_train, y_train)

# 7. Evaluate (MAE)
y_pred = model.predict(X_test)
train_pred = model.predict(X_train)

# Calculate Errors
test_mae = mean_absolute_error(y_test, y_pred)
train_mae = mean_absolute_error(y_train, train_pred)
mae_per_col = mean_absolute_error(y_test, y_pred, multioutput='raw_values')

print("\n" + "="*50)
print("📊 MODEL PERFORMANCE (Image Embeddings Only)")
print("="*50)
print(f"Train MAE:           {train_mae:.4f}")
print(f"Test MAE (Global):   {test_mae:.4f}")
print("-" * 50)
for i, col in enumerate(y_cols):
    print(f"MAE for {col:<25}: {mae_per_col[i]:.4f}")
print("="*50)

# 8. Verify Input Features
print("\nFeatures used in X (Should be z0-z31 + type columns):")
print(X.columns.tolist())

⚠️ Found NaNs in X (filling with 0)...
X Shape: (1479, 43)
y Shape: (1479, 3)

Training Image-Only Neural Network...

📊 MODEL PERFORMANCE (Image Embeddings Only)
Train MAE:           12.5340
Test MAE (Global):   15.6206
--------------------------------------------------
MAE for target_adhesion          : 24.6791
MAE for target_flank             : 14.2056
MAE for target_flank_adhesion    : 7.9770

Features used in X (Should be z0-z31 + type columns):
['type_adhesion', 'type_flank_wear', 'type_flank_wear+adhesion', 'z0', 'z1', 'z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11', 'z12', 'z13', 'z14', 'z15', 'z16', 'z17', 'z18', 'z19', 'z20', 'z21', 'z22', 'z23', 'z24', 'z25', 'z26', 'z27', 'z28', 'z29', 'z30', 'z31', 'n', 'Vf', 'material_CK45', 'Ae', 'fz', 'Ap', 'Vc', 'sample_index_scaled']
